In [1]:
# third-party
import pandas as pd
import numpy as np
import pickle

# local
from waveform import get_waveforms_by_id_by_activity
from get_data import get_data_by_id_activity
from constants import DEVICE_INFO

In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath, processing4visualization=True)

Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
amplitudes_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview.keys():
    for activity in overview[id].keys():
        for device in ['mag', 'pzt', 'airflow']:
            try:
                waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview[id][activity][DEVICE_INFO[device]["name"]])
                x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
                amplitudes_df = pd.concat([amplitudes_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)
            except Exception as e:
                print(e)
                print(id, activity, device)

amplitudes_df            

zero-size array to reduction operation maximum which has no identity
83J1 UAL pzt


,amplitude,activity,sensor,id
0,40.536193,SNB,MAG,7OYX
1,31.358371,SNB,MAG,7OYX
2,79.372017,SNB,MAG,7OYX
3,44.410013,SNB,MAG,7OYX
4,42.135056,SNB,MAG,7OYX
...,...,...,...,...
4798,3005.328737,SS,PZT,D4GQ
4799,1555.708966,SS,Airflow,D4GQ
4800,2008.474585,SS,Airflow,D4GQ
4801,1366.735212,SS,Airflow,D4GQ


In [4]:
amplitudes_df[amplitudes_df["sensor"]=='MAG'].max()

amplitude    330.55232
activity           UAR
sensor             MAG
id                Y6O3
dtype: object

In [5]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f}")
print(f"pk-to-pk for PZT sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f}")
print(f"pk-to-pk for Airflow sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f}\n")

print(f"coefficient of variation MAG: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.2f}")
print(f"coefficient of variation PZT: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.2f}")
print(f"coefficient of variation Airflow: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.2f}")

pk-to-pk for MAG sensor: 60.9 +/- 32.3
pk-to-pk for PZT sensor: 1455.7 +/- 1048.6
pk-to-pk for Airflow sensor: 1156.2 +/- 452.7

coefficient of variation MAG: 0.53
coefficient of variation PZT: 0.72
coefficient of variation Airflow: 0.39


In [6]:
amplitudes_by_activity_df = pd.DataFrame(columns=['Activity', 'Sensor', 'Pk-to-pk amplitude', 'std/mean'])

for activity in amplitudes_df['activity'].unique():
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'MAG')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'MAG', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)
    
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'PZT')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'PZT', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)

amplitudes_by_activity_df

,Activity,Sensor,Pk-to-pk amplitude,std/mean
0,SNB,MAG,$36.31 \pm 15.0$,0.41
1,SNB,PZT,$1040.31 \pm 676.0$,0.65
2,SGB,MAG,$77.74 \pm 34.0$,0.44
3,SGB,PZT,$1610.77 \pm 911.0$,0.57
4,MIXB,MAG,$47.59 \pm 21.0$,0.45
5,MIXB,PZT,$1508.11 \pm 840.0$,0.56
6,STNB,MAG,$42.87 \pm 21.0$,0.50
7,STNB,PZT,$1120.3 \pm 672.0$,0.60
8,MCH,MAG,$62.8 \pm 23.0$,0.37
9,MCH,PZT,$1678.91 \pm 798.0$,0.48


In [7]:
amplitudes_df

,amplitude,activity,sensor,id
0,40.536193,SNB,MAG,7OYX
1,31.358371,SNB,MAG,7OYX
2,79.372017,SNB,MAG,7OYX
3,44.410013,SNB,MAG,7OYX
4,42.135056,SNB,MAG,7OYX
...,...,...,...,...
4798,3005.328737,SS,PZT,D4GQ
4799,1555.708966,SS,Airflow,D4GQ
4800,2008.474585,SS,Airflow,D4GQ
4801,1366.735212,SS,Airflow,D4GQ


### Analysis on alternative positioning

In [8]:
amplitudes_middle_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview_middle.keys():
    for activity in overview_middle[id].keys():
        for device in ['mag', 'pzt']:
            waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview_middle[id][activity][DEVICE_INFO[device]["name"]])
            x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
            amplitudes_middle_df = pd.concat([amplitudes_middle_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)

amplitudes_middle_df            

,amplitude,activity,sensor,id
0,25.739095,SNBm,MAG,7OYX
1,22.545888,SNBm,MAG,7OYX
2,29.629096,SNBm,MAG,7OYX
3,31.519398,SNBm,MAG,7OYX
4,41.274917,SNBm,MAG,7OYX
...,...,...,...,...
826,100.321882,UARm,MAG,D4GQ
827,135.326500,UARm,MAG,D4GQ
828,1958.924378,UARm,PZT,D4GQ
829,2806.295001,UARm,PZT,D4GQ


In [9]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])}")
print(f"pk-to-pk for PZT sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])}")
print(f"pk-to-pf for Airflow: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])}")
print(np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']) / np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']))

pk-to-pk for MAG sensor: 63.26745015343785 +/- 38.451926738018656
pk-to-pk for PZT sensor: 1051.130084186841 +/- 938.4464718261065
pk-to-pf for Airflow: nan +/- nan
0.6077679224429632
